In [1]:
import numpy as np

In [69]:
input_shape = (1, 5, 5)
spatial_scalars = np.zeros([2, 5, 5])
spatial_scalars[0][0][0] = 1
spatial_scalars[0][2][1] = 1
spatial_scalars[0][2][3] = 0.67
spatial_scalars[1][0][0] = 1
spatial_scalars[1][2][1] = 0.67
spatial_scalars[1][2][3] = 1
print(spatial_scalars)
input = np.array([
    [[0.1, 0.2, 0.3, 0.4, 0.5],
     [0.6, 0.9, 0.6, 0.9, 0.6],
     [0.9, 0.0, 0.9, 0.0, 0.9],
     [0.6, 0.9, 0.6, 0.9, 0.6],
     [0.1, 0.2, 0.3, 0.4, 0.5]]
    ])
filters = np.random.random((2, 3, 3))

[[[1.   0.   0.   0.   0.  ]
  [0.   0.   0.   0.   0.  ]
  [0.   1.   0.   0.67 0.  ]
  [0.   0.   0.   0.   0.  ]
  [0.   0.   0.   0.   0.  ]]

 [[1.   0.   0.   0.   0.  ]
  [0.   0.   0.   0.   0.  ]
  [0.   0.67 0.   1.   0.  ]
  [0.   0.   0.   0.   0.  ]
  [0.   0.   0.   0.   0.  ]]]
12.9
12.9


In [3]:
print("spatial_scalars:\n", spatial_scalars)
print("input:\n", input)
print("convolutional filters:\n", filters)

spatial_scalars:
 [[[1.   0.   0.   0.   0.  ]
  [0.   0.   0.   0.   0.  ]
  [0.   1.   0.   0.67 0.  ]
  [0.   0.   0.   0.   0.  ]
  [0.   0.   0.   0.   0.  ]]

 [[1.   0.   0.   0.   0.  ]
  [0.   0.   0.   0.   0.  ]
  [0.   0.67 0.   1.   0.  ]
  [0.   0.   0.   0.   0.  ]
  [0.   0.   0.   0.   0.  ]]]
input:
 [[[0.1 0.2 0.3 0.4 0.5]
  [0.6 0.9 0.6 0.9 0.6]
  [0.9 0.  0.9 0.  0.9]
  [0.6 0.9 0.6 0.9 0.6]
  [0.1 0.2 0.3 0.4 0.5]]]
convolutional filters:
 [[[0.02883418 0.45232376 0.94250214]
  [0.99528437 0.77810749 0.73695356]
  [0.73952314 0.3275751  0.28513554]]

 [[0.69069447 0.50452381 0.5487601 ]
  [0.88158949 0.42305141 0.05459987]
  [0.17279914 0.05083774 0.97796674]]]


In [4]:
# Simply applies convolutional filter and returns the inner product
def filter_pass(domain, x, y, filter):
    acc = 0 # bias put here as a f(filter, x, y)
    offset = (filter.shape[1]//2) # assume conv filters are square
    for dx in range(-offset, offset + 1):
        for dy in range(-offset, offset + 1):
            acc += domain[0][(x + dx) % domain.shape[1]][(y + dy) % domain.shape[2]] * filter[dx + offset][dy + offset]
    return acc


## Spatially variant scalar convolution operation, wrap around, no bias
def svconv2d_filter(input, filter, spatial_scalars):
    ret = np.empty(shape=input.shape)
    for x in range(input.shape[1]):
        for y in range(input.shape[2]):
            # 0 here for now because we only are trying this out on 1 channel, will iterate over all channels when time comes.
            ret[0][x][y] = spatial_scalars[x][y] * filter_pass(input, x, y, filter)
    return ret

def svconv2d(input, filters, spatial_scalars):
    return np.array([svconv2d_filter(input, filter=filters[i], spatial_scalars=spatial_scalars[i]) for i in range(filters.shape[0])])

# after = svconv2d_filter(input, filters[0], spatial_scalars[0])
print('input:\n', input)
print('after:\n', svconv2d(input, filters, spatial_scalars))

input:
 [[[0.1 0.2 0.3 0.4 0.5]
  [0.6 0.9 0.6 0.9 0.6]
  [0.9 0.  0.9 0.  0.9]
  [0.6 0.9 0.6 0.9 0.6]
  [0.1 0.2 0.3 0.4 0.5]]]
after:
 [[[[1.86787447 0.         0.         0.         0.        ]
   [0.         0.         0.         0.         0.        ]
   [0.         3.45852012 0.         2.31720848 0.        ]
   [0.         0.         0.         0.         0.        ]
   [0.         0.         0.         0.         0.        ]]]


 [[[2.0139237  0.         0.         0.         0.        ]
   [0.         0.         0.         0.         0.        ]
   [0.         1.86027383 0.         2.7765281  0.        ]
   [0.         0.         0.         0.         0.        ]
   [0.         0.         0.         0.         0.        ]]]]


In [5]:
1.83395446/2.73724546

0.6700000006575954

In [6]:
## Circular padding:
import torch
import torch.nn.functional as F
input = torch.ones((1, 2, 28, 28))
print(input.shape)
print(F.pad(input, (1,1,1,1),"circular").shape)

torch.Size([1, 2, 28, 28])
torch.Size([1, 2, 30, 30])


In [24]:
inpt = torch.rand(1, 2, 3, 3)
a = F.pad(inpt, (1, 1, 1, 1), mode='circular')
conv = torch.rand(1, 2, 3, 3)

spatial_scalars = torch.zeros(2, 3, 3)

# print(F.conv2d(a, conv))

output = torch.zeros(inpt.shape)
for chan in range(a.shape[1]):
    intermed = F.conv2d(a[:,chan:chan+1,:,:], conv[:,chan:chan+1,:,:])
    print(intermed)
    output[:,chan:chan+1,:,:] += 1 * intermed
print(output[:,0,:,:] + output[:,1,:,:])
print(F.conv2d(a, conv))
assert not (torch.isclose(output[:,0,:,:] + output[:,1,:,:], F.conv2d(a, conv)).__contains__(False))

tensor([[[[2.2539, 2.2681, 2.3813],
          [2.6034, 2.4541, 2.3977],
          [2.2744, 2.1757, 2.3888]]]])
tensor([[[[2.8652, 2.9157, 2.8558],
          [2.7867, 2.8869, 2.5197],
          [2.5025, 3.0357, 2.6416]]]])
tensor([[[5.1191, 5.1838, 5.2371],
         [5.3901, 5.3410, 4.9174],
         [4.7769, 5.2114, 5.0305]]])
tensor([[[[5.1191, 5.1838, 5.2371],
          [5.3901, 5.3410, 4.9174],
          [4.7769, 5.2114, 5.0305]]]])


In [8]:
x = torch.randint(0, 9, (1, 2, 2, 2))
print(x)
a = torch.zeros((1, 2, 2, 2))
a[0][0][1][0] = 1
a[0][1][1][0] = 3
print(a)
print((a * x).sum(dim=1))

tensor([[[[4, 2],
          [3, 3]],

         [[7, 8],
          [3, 0]]]])
tensor([[[[0., 0.],
          [1., 0.]],

         [[0., 0.],
          [3., 0.]]]])
tensor([[[ 0.,  0.],
         [12.,  0.]]])


In [9]:
(1, 1) + (2,)

(1, 1, 2)

In [10]:
from svconv import SVConv2d
import torch
import torch.nn.functional as F

input = torch.rand(1, 2, 3, 3)

convlayer = SVConv2d(in_channels=2, out_channels=2, kernel_size=3, spatial_scalar_hint=input.size(), stride=1, padding=(1,1), padding_mode='circular')
convlayer(input)

tensor([[[[0.1916, 0.0000, 0.0000],
          [0.0000, 0.0000, 0.0000],
          [0.0000, 0.0000, 0.0000]],

         [[0.1755, 0.0000, 0.0000],
          [0.0000, 0.0000, 0.0000],
          [0.0000, 0.0000, 0.0000]]]], grad_fn=<AddcmulBackward0>)

In [11]:
import torch
import torch.nn as nn
from torch.nn import functional as F

torch.manual_seed(1234)

convlayer = nn.Conv2d(2, 2, 3, padding=(1,1), padding_mode='circular', bias=False)

input = torch.rand(2, 2, 3, 3)

padded_input = F.pad(input, convlayer._reversed_padding_repeated_twice, mode=convlayer.padding_mode)
output = F.conv2d(padded_input, convlayer.weight, convlayer.bias, convlayer.stride, torch.nn.modules.utils._pair(0), convlayer.dilation, convlayer.groups)

output11 = F.conv2d(padded_input[0:1,0:1,:,:], convlayer.weight[:,0:1,:,:], convlayer.bias, convlayer.stride, torch.nn.modules.utils._pair(0), convlayer.dilation, convlayer.groups)
output12 = F.conv2d(padded_input[0:1,1:2,:,:], convlayer.weight[:,1:2,:,:], convlayer.bias, convlayer.stride, torch.nn.modules.utils._pair(0), convlayer.dilation, convlayer.groups)

output21 = F.conv2d(padded_input[1:2,0:1,:,:], convlayer.weight[:,0:1,:,:], convlayer.bias, convlayer.stride, torch.nn.modules.utils._pair(0), convlayer.dilation, convlayer.groups)
output22 = F.conv2d(padded_input[1:2,1:2,:,:], convlayer.weight[:,1:2,:,:], convlayer.bias, convlayer.stride, torch.nn.modules.utils._pair(0), convlayer.dilation, convlayer.groups)

print(input)

print(convlayer.weight)

print(output11 + output12)
print(output21 + output22)
print(convlayer(input))

# print(convlayer.weight)
# print(convlayer.bias)

tensor([[[[0.8436, 0.4265, 0.9561],
          [0.0770, 0.4108, 0.0014],
          [0.5414, 0.6419, 0.2976]],

         [[0.7077, 0.4189, 0.0655],
          [0.8839, 0.8083, 0.7528],
          [0.8988, 0.6839, 0.7658]]],


        [[[0.9149, 0.3993, 0.1100],
          [0.2541, 0.4333, 0.4451],
          [0.4966, 0.7865, 0.6604]],

         [[0.1303, 0.3498, 0.3824],
          [0.8043, 0.3186, 0.2908],
          [0.4196, 0.3728, 0.3769]]]])
Parameter containing:
tensor([[[[-0.2220, -0.0462, -0.1132],
          [-0.0629, -0.2082,  0.0946],
          [-0.2113, -0.0150,  0.0819]],

         [[-0.0794,  0.1337,  0.0297],
          [ 0.1296,  0.1512, -0.1041],
          [ 0.0857, -0.1020,  0.0739]]],


        [[[-0.1232,  0.1090,  0.0477],
          [-0.0922, -0.1156,  0.0610],
          [ 0.2199,  0.1131, -0.0228]],

         [[-0.0114,  0.1340, -0.1638],
          [ 0.0784, -0.0781,  0.1364],
          [-0.0841,  0.0117,  0.0796]]]], requires_grad=True)
tensor([[[[-0.1413, -0.0174, -0.3211

In [12]:
print(padded_input)

tensor([[[[0.2976, 0.5414, 0.6419, 0.2976, 0.5414],
          [0.9561, 0.8436, 0.4265, 0.9561, 0.8436],
          [0.0014, 0.0770, 0.4108, 0.0014, 0.0770],
          [0.2976, 0.5414, 0.6419, 0.2976, 0.5414],
          [0.9561, 0.8436, 0.4265, 0.9561, 0.8436]],

         [[0.7658, 0.8988, 0.6839, 0.7658, 0.8988],
          [0.0655, 0.7077, 0.4189, 0.0655, 0.7077],
          [0.7528, 0.8839, 0.8083, 0.7528, 0.8839],
          [0.7658, 0.8988, 0.6839, 0.7658, 0.8988],
          [0.0655, 0.7077, 0.4189, 0.0655, 0.7077]]],


        [[[0.6604, 0.4966, 0.7865, 0.6604, 0.4966],
          [0.1100, 0.9149, 0.3993, 0.1100, 0.9149],
          [0.4451, 0.2541, 0.4333, 0.4451, 0.2541],
          [0.6604, 0.4966, 0.7865, 0.6604, 0.4966],
          [0.1100, 0.9149, 0.3993, 0.1100, 0.9149]],

         [[0.3769, 0.4196, 0.3728, 0.3769, 0.4196],
          [0.3824, 0.1303, 0.3498, 0.3824, 0.1303],
          [0.2908, 0.8043, 0.3186, 0.2908, 0.8043],
          [0.3769, 0.4196, 0.3728, 0.3769, 0.4196],
    

In [13]:
from torch import Tensor, tensor

def compute_cross_corr(t1: Tensor, t2: Tensor):
    assert t1.numel() == t2.numel(), "{} {} dimension do not match".format(t1.shape, t2.shape)
    a = torch.mul(t1, t2)
    return torch.sum(a)

In [14]:
compute_cross_corr(padded_input[0:1,0:1,0:3,0:3], convlayer.weight[0:1,0:1,:,:])

tensor(-0.3270, grad_fn=<SumBackward0>)

In [15]:
print(compute_cross_corr(padded_input[0:1,1:2,0:3,0:3], convlayer.weight[0:1,0:1,:,:]))

tensor(-0.5070, grad_fn=<SumBackward0>)


In [26]:
-0.1070
padded_input[0:1,1:2,0:3,0:3]

tensor([[[[0.7658, 0.8988, 0.6839],
          [0.0655, 0.7077, 0.4189],
          [0.7528, 0.8839, 0.8083]]]])

In [55]:
import torch
import torch.nn as nn
from torch.nn import functional as F

torch.manual_seed(1234)

convlayer = nn.Conv2d(2, 3, 3, padding=(1,1), padding_mode='circular', bias=False)

input = torch.rand(2, 2, 1, 1)

padded_input = F.pad(input, convlayer._reversed_padding_repeated_twice, mode=convlayer.padding_mode)
output = F.conv2d(padded_input, convlayer.weight, convlayer.bias, convlayer.stride, torch.nn.modules.utils._pair(0), convlayer.dilation, convlayer.groups)

output11 = F.conv2d(padded_input[0:1,0:1,:,:], convlayer.weight[:,0:1,:,:], convlayer.bias, convlayer.stride, torch.nn.modules.utils._pair(0), convlayer.dilation, convlayer.groups)
output12 = F.conv2d(padded_input[0:1,1:2,:,:], convlayer.weight[:,1:2,:,:], convlayer.bias, convlayer.stride, torch.nn.modules.utils._pair(0), convlayer.dilation, convlayer.groups)

output21 = F.conv2d(padded_input[1:2,0:1,:,:], convlayer.weight[:,0:1,:,:], convlayer.bias, convlayer.stride, torch.nn.modules.utils._pair(0), convlayer.dilation, convlayer.groups)
output22 = F.conv2d(padded_input[1:2,1:2,:,:], convlayer.weight[:,1:2,:,:], convlayer.bias, convlayer.stride, torch.nn.modules.utils._pair(0), convlayer.dilation, convlayer.groups)

print(input)

print(convlayer.weight)

print(output11 + output12)
print(output21 + output22)
print(convlayer(input))

# print(convlayer.weight)
# print(convlayer.bias)

tensor([[[[0.9149]],

         [[0.3993]]],


        [[[0.1100]],

         [[0.2541]]]])
Parameter containing:
tensor([[[[-0.2220, -0.0462, -0.1132],
          [-0.0629, -0.2082,  0.0946],
          [-0.2113, -0.0150,  0.0819]],

         [[-0.0794,  0.1337,  0.0297],
          [ 0.1296,  0.1512, -0.1041],
          [ 0.0857, -0.1020,  0.0739]]],


        [[[-0.1232,  0.1090,  0.0477],
          [-0.0922, -0.1156,  0.0610],
          [ 0.2199,  0.1131, -0.0228]],

         [[-0.0114,  0.1340, -0.1638],
          [ 0.0784, -0.0781,  0.1364],
          [-0.0841,  0.0117,  0.0796]]],


        [[[ 0.1620, -0.0346,  0.2150],
          [-0.1994, -0.0420, -0.2350],
          [ 0.0195,  0.0669, -0.0954]],

         [[ 0.0979, -0.0382, -0.2048],
          [ 0.1810,  0.1453,  0.1192],
          [ 0.1880,  0.0867,  0.1253]]]], requires_grad=True)
tensor([[[[-0.5155]],

         [[ 0.2212]],

         [[ 0.1487]]]], grad_fn=<AddBackward0>)
tensor([[[[0.0036]],

         [[0.0477]],

         [

In [59]:
print(convlayer.weight[0,0,:,:].sum() * 0.9149)
print(convlayer.weight[1,0,:,:].sum() * 0.9149)
print(convlayer.weight[2,0,:,:].sum() * 0.9149)

print(convlayer.weight[0,1,:,:].sum() * 0.3993)
print(convlayer.weight[1,1,:,:].sum() * 0.3993)
print(convlayer.weight[2,1,:,:].sum() * 0.3993)


print('batch next')
print(convlayer.weight[0,0,:,:].sum() * 0.1100)
print(convlayer.weight[1,0,:,:].sum() * 0.1100)
print(convlayer.weight[2,0,:,:].sum() * 0.1100)

print(convlayer.weight[0,1,:,:].sum() * 0.2541)
print(convlayer.weight[1,1,:,:].sum() * 0.2541)
print(convlayer.weight[2,1,:,:].sum() * 0.2541)

tensor(-0.6426, grad_fn=<MulBackward0>)
tensor(0.1802, grad_fn=<MulBackward0>)
tensor(-0.1309, grad_fn=<MulBackward0>)
tensor(0.1271, grad_fn=<MulBackward0>)
tensor(0.0409, grad_fn=<MulBackward0>)
tensor(0.2796, grad_fn=<MulBackward0>)
batch next
tensor(-0.0773, grad_fn=<MulBackward0>)
tensor(0.0217, grad_fn=<MulBackward0>)
tensor(-0.0157, grad_fn=<MulBackward0>)
tensor(0.0809, grad_fn=<MulBackward0>)
tensor(0.0260, grad_fn=<MulBackward0>)
tensor(0.1780, grad_fn=<MulBackward0>)


In [67]:
import torch
import torch.nn as nn
from torch.nn import functional as F

torch.manual_seed(1234)

convlayer = nn.Conv2d(2, 3, 3, padding=(1,1), padding_mode='circular', bias=False)

input = torch.rand(2, 2, 1, 1)

padded_input = F.pad(input, convlayer._reversed_padding_repeated_twice, mode=convlayer.padding_mode)
output = F.conv2d(padded_input, convlayer.weight, convlayer.bias, convlayer.stride, torch.nn.modules.utils._pair(0), convlayer.dilation, convlayer.groups)

output11 = F.conv2d(padded_input[0:1,0:1,:,:], convlayer.weight[:,0:1,:,:], convlayer.bias, convlayer.stride, torch.nn.modules.utils._pair(0), convlayer.dilation, convlayer.groups)
output12 = F.conv2d(padded_input[0:1,1:2,:,:], convlayer.weight[:,1:2,:,:], convlayer.bias, convlayer.stride, torch.nn.modules.utils._pair(0), convlayer.dilation, convlayer.groups)

output21 = F.conv2d(padded_input[1:2,0:1,:,:], convlayer.weight[:,0:1,:,:], convlayer.bias, convlayer.stride, torch.nn.modules.utils._pair(0), convlayer.dilation, convlayer.groups)
output22 = F.conv2d(padded_input[1:2,1:2,:,:], convlayer.weight[:,1:2,:,:], convlayer.bias, convlayer.stride, torch.nn.modules.utils._pair(0), convlayer.dilation, convlayer.groups)

# print(input)

# print(convlayer.weight)

print(output11 * 1)
print(output12 * 1)

print('batch next')
print(output21)
print(output22)

# print(convlayer(input))

# print(convlayer.weight)
# print(convlayer.bias)

tensor([[[[-0.6426]],

         [[ 0.1802]],

         [[-0.1310]]]], grad_fn=<MulBackward0>)
tensor([[[[0.1271]],

         [[0.0409]],

         [[0.2796]]]], grad_fn=<MulBackward0>)
batch next
tensor([[[[-0.0773]],

         [[ 0.0217]],

         [[-0.0158]]]], grad_fn=<SlowConv2DBackward0>)
tensor([[[[0.0809]],

         [[0.0260]],

         [[0.1780]]]], grad_fn=<SlowConv2DBackward0>)
